In [1]:
from sklearn.metrics import roc_auc_score
import torch
from torchvision.transforms import Compose, Normalize, Resize, ToTensor
from torch.optim import SGD
from torch.nn import BCELoss
import torch.utils.model_zoo as model_zoo
from cxrlib.models import guan_resnet_ag
from cxrlib.read_data import ChestXrayDataSet
from cxrlib.results import compute_AUCs

In [2]:
cur = 17

In [3]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']

In [4]:
log = open("auc.txt","a")
model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

In [5]:
model = guan_resnet_ag.GuanResNet50_TwoAG().cuda()
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load(str(cur)))

In [6]:
batch_n = 0
normalize = Normalize([0.485, 0.456, 0.406],
                  [0.229, 0.224, 0.225])
test_dataset = ChestXrayDataSet('/media/minh/UStorage/chestxray14/images',
                            '/media/minh/UStorage/chestxray14/labels/test_list.processed',
                            transform=Compose([Resize(224), ToTensor(), normalize]))
loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)
gt = torch.FloatTensor().cuda()
pred = torch.FloatTensor().cuda() 
model.eval()

for i in range(1,2):
    for inp, target in loader:
        target = torch.autograd.Variable(target).cuda()
        inp = torch.autograd.Variable(inp).cuda()
        out = model(inp)
        # Add results of the model's output to the aggregated prediction vector, and also add aggregated
        # ground truth information as well
        pred = torch.cat((pred, out.data), 0)
        gt = torch.cat((gt, target.data), 0)
        #print(batch_n)
        #if batch_n > max_batches:
        #    break
        batch_n += 1

    # Compute the model area under curve (AUC).
    auc = compute_AUCs(gt, pred)
    print("AUC Results: {}".format(sum(auc) / len(auc)))
    log.write("\n============\n")
    log.write(str(sum(auc)/len(auc)))
    log.write('\n'+str(cur))
    log.write("\n============\n")
    del test_dataset
    del loader
    del model
    del gt
    del pred

/home/minh/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/minh/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/minh/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


AUC Results: 0.678778165100956


In [7]:
auc

[0.6768438397061782,
 0.684170587973476,
 0.7345902207917128,
 0.6520573062499395,
 0.5915761805124292,
 0.5779218811551164,
 0.614153940690421,
 0.7133103282438882,
 0.6896306088743902,
 0.7903370737280997,
 0.6445943900888734,
 0.7076637945881398,
 0.659505500920061,
 0.7665386578906586]

In [8]:
0.6837054404065707

0.6837054404065707

In [9]:
0.6867063861304776

0.6867063861304776